In [1]:
from __future__ import print_function

import torch 
import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import numpy as np
import pandas as pd
import re
import operator

import os

Idx = str("0")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=Idx

from tqdm.notebook import tqdm
import utils
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from transformers import *

Tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
You are using torchaudio==0.8.1, but torchaudio>=0.10.0 is required to use MCTCTFeatureExtractor. This requires torch>=1.10.0. Please upgrade torch and torchaudio.
You are using torch==1.8.1+cu111, but torch>=1.10.0 is required to use ViltModel. Please upgrade torch.
loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa

In [3]:
import utils
from imp import reload
reload(utils)

x_train, y_train, x_valid, y_valid, x_test, y_test, TopicList, Idx2Topic = utils.LoadDatasets("TREC")
Idx2Topic_list = []
for i in range(len(set(TopicList))): Idx2Topic_list.append(Idx2Topic[i])
Idx2Topic_list = np.array(Idx2Topic_list)

if not len(x_valid):
    from sklearn.model_selection import train_test_split
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

Found cached dataset trec (/home/jovyan/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import *

MAX_SEQ_LEN = 512

CLS_id = Tokenizer.convert_tokens_to_ids(["[CLS]"])
SEP_id = Tokenizer.convert_tokens_to_ids(["[SEP]"])
PAD_id = Tokenizer.convert_tokens_to_ids(["[PAD]"])

pbar = tqdm(total = len(x_train))
x_train_emb, x_valid_emb, x_test_emb = [], [], []
for x in x_train:
    pbar.update(1)
    Tokens = Tokenizer.encode(x, add_special_tokens=False)[:MAX_SEQ_LEN-2]
    x_train_emb.append(CLS_id + Tokens + SEP_id + PAD_id*(MAX_SEQ_LEN-2-len(Tokens)))
pbar.close()

pbar = tqdm(total = len(x_valid))
for x in x_valid:
    pbar.update(1)
    Tokens = Tokenizer.encode(x, add_special_tokens=False)[:MAX_SEQ_LEN-2]
    x_valid_emb.append(CLS_id + Tokens + SEP_id + PAD_id*(MAX_SEQ_LEN-2-len(Tokens)))
pbar.close()

pbar = tqdm(total = len(x_test))
for x in x_test:
    pbar.update(1)
    Tokens = Tokenizer.encode(x, add_special_tokens=False)[:MAX_SEQ_LEN-2]
    x_test_emb.append(CLS_id + Tokens + SEP_id + PAD_id*(MAX_SEQ_LEN-2-len(Tokens)))
pbar.close()

x_train_emb, x_valid_emb, x_test_emb = np.array(x_train_emb), np.array(x_valid_emb), np.array(x_test_emb)
y_train, y_valid, y_test = np.asarray(y_train), np.asarray(y_valid), np.asarray(y_test)

  0%|          | 0/4634 [00:00<?, ?it/s]

  0%|          | 0/818 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [5]:
NumClass = max(len(set(y_train)), max(y_train)+1)
print(x_train_emb.shape, x_valid_emb.shape, x_test_emb.shape)
print("NumClass", NumClass)

(4634, 512) (818, 512) (500, 512)
NumClass 6


In [6]:
import dataloader
import copy
import random

### Hyperparameters
Hyperparams = {
    "NumClfEpoch" : 100,
    "EmbeddingSize" : 768, # 300, 768
    "BatchSize" : 2**4,
    "LearningRate_EMB" : 0, # Freeze
    "LearningRate_CLF" : 1e-3,
    "MaxSeqLen" : MAX_SEQ_LEN,
    "NumClass" : NumClass,
}

train_dataset = dataloader.ClassifyDataset(x=x_train_emb, y=y_train)
valid_dataset = dataloader.ClassifyDataset(x=x_valid_emb, y=y_valid)
test_dataset = dataloader.ClassifyDataset(x=x_test_emb, y=y_test)
train_loader = DataLoader(dataset=train_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)

In [7]:
class LinearClassifier(nn.Module):
    def __init__(self, Hyperparams):
        ###
        super(LinearClassifier, self).__init__()
        self.EmbeddingSize = Hyperparams["EmbeddingSize"]
        self.LearningRate = Hyperparams["LearningRate_CLF"]
        self.BatchSize = Hyperparams["BatchSize"]
        self.MaxSeqLen = Hyperparams["MaxSeqLen"]
        self.NumClass = Hyperparams["NumClass"]
        
        self.fc = nn.Sequential(
#             nn.Dropout(self.DropoutRate),
            nn.Linear(self.EmbeddingSize, self.NumClass),
        )
        
    def forward(self, x):
        out = self.fc(x)
        return out

In [11]:
for Iter in range(1):
    Hyperparams["LearningRate_EMB"] = 0
    Hyperparams["LearningRate_CLF"] = 1e-3
    
#     Embedding = AutoModel.from_pretrained('bert-base-uncased')
#     Embedding = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
#     Embedding = AutoModel.from_pretrained('kanishka/GlossBERT')
#     Embedding = AutoModel.from_pretrained('./defbert')
    Embedding = AutoModel.from_pretrained('/home/jovyan/temp/defbert1')

    Embedding = nn.DataParallel(Embedding).to(device)
    Classifier = LinearClassifier(Hyperparams)
    Classifier = nn.DataParallel(Classifier).to(device)
    
    criterion = nn.CrossEntropyLoss()
    UNKWords = []

    # Train the model
    maxacc_all = 0; Step = 0
    
    optimizer_emb = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate_EMB"])
    optimizer_clf = torch.optim.Adam(filter(lambda p: p.requires_grad, Classifier.parameters()), lr=Hyperparams["LearningRate_CLF"])
    maxacc_val = 0
    EarlyStopCnt = 5
    total_step = len(train_loader)
    pbar1 = tqdm(total = Hyperparams["NumClfEpoch"], leave=False, desc="Epoch")
#         Embedding.train() # train mode
    Classifier.train()
    for epoch in range(Hyperparams["NumClfEpoch"]):
        pbar1.update(1)
        pbar2 = tqdm(total = len(train_dataset), leave=False, desc="Training")
        for i, (texts, labels) in enumerate(train_loader):
            texts = texts.to(device)
            labels = labels.to(device)
            # Forward pass
            emb = Embedding(texts, output_hidden_states=True)
            outputs = Classifier(emb[2][-1][:,0,:])
            loss = criterion(outputs, labels)
            optimizer_emb.zero_grad()
            optimizer_clf.zero_grad()
            loss.backward()
            optimizer_emb.step()
            optimizer_clf.step()
            pbar2.update(texts.size(0))
        pbar2.close()

        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, Hyperparams["NumClfEpoch"], loss.item()), end=' ')

        # Valid the model
        Embedding.eval()
        Classifier.eval()
        with torch.no_grad():
            correct = 0.
            total = 0
            for texts, labels in valid_loader:
                texts = texts.to(device)
                labels = labels.to(device)
                emb = Embedding(texts, output_hidden_states=True)
                outputs = Classifier(emb[2][-1][:,0,:])
                predicted = []
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            if (correct/total) >= maxacc_val:
                maxacc_val = correct/total
                torch.save(Embedding.state_dict(), "./save/Embedding"+ Idx)
                torch.save(Classifier.state_dict(), "./save/Classifier"+ Idx)
                EarlyStopCnt = 5

            print('ValidAcc: {:.4f} % , MaxAcc: {:.4f}, EarlyStopCnt: {}'.format(100 * correct/total, maxacc_val, EarlyStopCnt))

            if (correct/total) < maxacc_val:
                EarlyStopCnt = EarlyStopCnt-1
                if EarlyStopCnt == 0:
                    break

    ### Test Acc.
    Embedding.load_state_dict(torch.load("./save/Embedding"+ Idx))
    Classifier.load_state_dict(torch.load("./save/Classifier"+ Idx))

    with torch.no_grad():
        Embedding.eval()
        Classifier.eval()
        correct = 0.
        total = 0
        for texts, labels in test_loader:
            texts = texts.to(device)
            labels = labels.to(device)
            emb = Embedding(texts, output_hidden_states=True)
            outputs = Classifier(emb[2][-1][:,0,:])
            predicted = []
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('TestAcc: {:.4f} % , ValidAcc: {:.4f}'.format(100 * correct / total, maxacc_val))

    pbar1.close()


loading configuration file /home/jovyan/temp/defbert1/config.json
Model config BertConfig {
  "_name_or_path": "/home/jovyan/temp/defbert1",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/jovyan/temp/defbert1/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at /home/jovyan/temp/defbert1.
If y

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/4634 [00:00<?, ?it/s]

Epoch [1/100], Loss: 1.5673 ValidAcc: 41.8093 % , MaxAcc: 0.4181, EarlyStopCnt: 5


Training:   0%|          | 0/4634 [00:00<?, ?it/s]

...

Epoch [34/100], Loss: 1.0126 ValidAcc: 74.0831 % , MaxAcc: 0.7555, EarlyStopCnt: 3


Training:   0%|          | 0/4634 [00:00<?, ?it/s]

Epoch [35/100], Loss: 0.6534 ValidAcc: 74.6944 % , MaxAcc: 0.7555, EarlyStopCnt: 2


Training:   0%|          | 0/4634 [00:00<?, ?it/s]

Epoch [36/100], Loss: 0.6383 ValidAcc: 74.9389 % , MaxAcc: 0.7555, EarlyStopCnt: 1
TestAcc: 80.4000 % , ValidAcc: 0.7555
